# Main Comparison and Analysis Dashboard

This notebook imports and compares results from all previous analysis notebooks, providing a comprehensive overview and comparison of all methods across different tasks.

In [43]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import json
from typing import Dict, List, Tuple
import glob
from pathlib import Path
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

In [44]:
# Configuration
VERSION = "1.00"
OUTPUT_PATH = "output"
RESULTS_PATH = os.path.join(OUTPUT_PATH, VERSION)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [45]:
def save_plot(plot_name: str, suffix: str = "", plot_path: str = "comparison") -> None:
    """Save current matplotlib figure."""
    timestamp: str = ""
    base_dir: str = os.path.join(OUTPUT_PATH, VERSION, plot_path)
    os.makedirs(base_dir, exist_ok=True)

    filename: str = f"{plot_name}_{suffix}_v{VERSION}_{timestamp}.png" if suffix else f"{plot_name}_v{VERSION}_{timestamp}.png"
    filepath: str = os.path.join(base_dir, filename)

    plt.savefig(filepath, bbox_inches="tight", dpi=300)
    print(f"Plot saved: {filepath}")

def save_dataframe(df: pd.DataFrame, name: str, suffix: str = "") -> None:
    """Save a DataFrame to CSV."""
    timestamp: str = ""
    base_dir: str = os.path.join(OUTPUT_PATH, VERSION)
    os.makedirs(base_dir, exist_ok=True)

    filename: str = f"{name}_{suffix}_v{VERSION}_{timestamp}.csv" if suffix else f"{name}_v{VERSION}_{timestamp}.csv"
    filepath: str = os.path.join(base_dir, filename)

    df.to_csv(filepath, index=True)
    print(f"Data saved: {filepath}")

## Load All Results

In [46]:
def load_csv_results(pattern: str) -> pd.DataFrame:
    """Load CSV results matching a pattern."""
    files = glob.glob(os.path.join(RESULTS_PATH, pattern))
    if files:
        # Get the most recent file
        latest_file = max(files, key=os.path.getctime)
        print(f"Loading: {latest_file}")
        return pd.read_csv(latest_file, index_col=0)
    else:
        print(f"No files found matching pattern: {pattern}")
        return pd.DataFrame()

def load_json_summary(pattern: str) -> dict:
    """Load JSON summary matching a pattern."""
    files = glob.glob(os.path.join(RESULTS_PATH, pattern))
    if files:
        latest_file = max(files, key=os.path.getctime)
        print(f"Loading: {latest_file}")
        with open(latest_file, 'r') as f:
            return json.load(f)
    else:
        print(f"No files found matching pattern: {pattern}")
        return {}

# Load results from each notebook
print("=== Loading Results from Previous Notebooks ===")

# Classification results
classification_results = load_csv_results("classification_metrics_multiclass_*.csv")
classification_summary = load_json_summary("classification_summary_*.json")

# Anomaly detection results
anomaly_results = load_csv_results("anomaly_detection_metrics_anomaly_*.csv")
anomaly_summary = load_json_summary("anomaly_detection_summary_*.json")

# Merged fault results
merged_results = load_csv_results("merged_fault_classification_metrics_merging_*.csv")
merged_summary = load_json_summary("merged_fault_analysis_summary_*.json")

# EDA results (if available)
eda_summary = load_json_summary("eda_summary_*.json")

print("\n=== Results Loading Complete ===")

=== Loading Results from Previous Notebooks ===
No files found matching pattern: classification_metrics_multiclass_*.csv
Loading: output/1.00/classification_summary_v1.00_.json
No files found matching pattern: anomaly_detection_metrics_anomaly_*.csv
Loading: output/1.00/anomaly_detection_summary_v1.00_.json
No files found matching pattern: merged_fault_classification_metrics_merging_*.csv
Loading: output/1.00/merged_fault_analysis_summary_v1.00_.json
No files found matching pattern: eda_summary_*.json

=== Results Loading Complete ===


## Data Overview

In [47]:
# Create data overview
data_overview = {
    'Analysis Type': ['Multi-class Classification', 'Binary Anomaly Detection', 'Merged Fault Classification'],
    'Number of Models': [
        len(classification_summary.get('models_trained', [])) if classification_summary else 0,
        len(anomaly_summary.get('models_trained', [])) if anomaly_summary else 0,
        len(merged_summary.get('models_trained', [])) if merged_summary else 0
    ],
    'Test Samples': [
        classification_summary.get('total_test_samples', 0) if classification_summary else 0,
        anomaly_summary.get('total_test_samples', 0) if anomaly_summary else 0,
        merged_summary.get('total_test_samples', 0) if merged_summary else 0
    ],
    'Number of Classes': [
        classification_summary.get('num_classes', 0) if classification_summary else 0,
        2,  # Binary classification
        merged_summary.get('num_classes', 0) if merged_summary else 0
    ],
    'Best Model': [
        classification_summary.get('best_model', 'N/A') if classification_summary else 'N/A',
        anomaly_summary.get('best_model', 'N/A') if anomaly_summary else 'N/A',
        merged_summary.get('best_model', 'N/A') if merged_summary else 'N/A'
    ],
    'Best Accuracy': [
        f"{classification_summary.get('best_accuracy', 0):.3f}" if classification_summary else 'N/A',
        f"{anomaly_summary.get('best_accuracy', 0):.3f}" if anomaly_summary else 'N/A',
        f"{merged_summary.get('best_accuracy', 0):.3f}" if merged_summary else 'N/A'
    ]
}

overview_df = pd.DataFrame(data_overview)
save_dataframe(overview_df, "analysis_overview", "comparison")

print("=== Tennessee Eastman Process Analysis Overview ===")
print(tabulate(overview_df, headers="keys", tablefmt="grid"))

Data saved: output/1.00/analysis_overview_comparison_v1.00_.csv
=== Tennessee Eastman Process Analysis Overview ===
+----+-----------------------------+--------------------+----------------+---------------------+---------------+-----------------+
|    | Analysis Type               |   Number of Models |   Test Samples |   Number of Classes | Best Model    |   Best Accuracy |
+====+=============================+====================+================+=====================+===============+=================+
|  0 | Multi-class Classification  |                  3 |          40320 |                   0 | Random Forest |           0.594 |
+----+-----------------------------+--------------------+----------------+---------------------+---------------+-----------------+
|  1 | Binary Anomaly Detection    |                  7 |          40320 |                   2 | MCUSUM        |           0.954 |
+----+-----------------------------+--------------------+----------------+---------------------+--

## Model Performance Comparison Across Tasks

In [48]:
def prepare_comparison_data():
    """Prepare data for cross-task model comparison."""
    comparison_data = []
    
    # Classification results
    if not classification_results.empty:
        for _, row in classification_results.iterrows():
            comparison_data.append({
                'Task': 'Multi-class Classification',
                'Model': row['Model'],
                'Accuracy': row['Accuracy'],
                'F1_Score': row.get('Macro_F1', row.get('F1-Score', np.nan)),
                'Precision': row.get('Macro_Precision', row.get('Precision', np.nan)),
                'Recall': row.get('Macro_Recall', row.get('Recall', np.nan))
            })
    
    # Anomaly detection results
    if not anomaly_results.empty:
        for _, row in anomaly_results.iterrows():
            comparison_data.append({
                'Task': 'Anomaly Detection',
                'Model': row['Model'],
                'Accuracy': row['Accuracy'],
                'F1_Score': row['F1-Score'],
                'Precision': row['Precision'],
                'Recall': row['Recall / TPR']
            })
    
    # Merged fault results
    if not merged_results.empty:
        for _, row in merged_results.iterrows():
            comparison_data.append({
                'Task': 'Merged Fault Classification',
                'Model': row['Model'],
                'Accuracy': row['Accuracy'],
                'F1_Score': row['Macro_F1'],
                'Precision': row['Macro_Precision'],
                'Recall': row['Macro_Recall']
            })
    
    return pd.DataFrame(comparison_data)

comparison_df = prepare_comparison_data()
save_dataframe(comparison_df, "cross_task_comparison", "comparison")

if not comparison_df.empty:
    print("\n=== Cross-Task Model Performance Comparison ===")
    print(tabulate(comparison_df, headers="keys", tablefmt="grid", floatfmt=".3f"))
else:
    print("No comparison data available.")

Data saved: output/1.00/cross_task_comparison_comparison_v1.00_.csv
No comparison data available.


## Comprehensive Visualization Dashboard

In [49]:
# 1. Accuracy comparison across all tasks
if not comparison_df.empty:
    plt.figure(figsize=(15, 8))
    
    # Pivot data for better visualization
    pivot_accuracy = comparison_df.pivot(index='Model', columns='Task', values='Accuracy')
    
    # Create grouped bar plot
    ax = pivot_accuracy.plot(kind='bar', width=0.8)
    plt.title('Model Accuracy Comparison Across All Tasks', fontsize=16, fontweight='bold')
    plt.xlabel('Model', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(title='Task', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.ylim(0, 1.05)
    
    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', rotation=90, fontsize=9)
    
    plt.tight_layout()
    save_plot('accuracy_comparison_all_tasks')
    plt.show()

In [50]:
# 2. F1-Score comparison across all tasks
if not comparison_df.empty:
    plt.figure(figsize=(15, 8))
    
    pivot_f1 = comparison_df.pivot(index='Model', columns='Task', values='F1_Score')
    
    ax = pivot_f1.plot(kind='bar', width=0.8)
    plt.title('Model F1-Score Comparison Across All Tasks', fontsize=16, fontweight='bold')
    plt.xlabel('Model', fontsize=12)
    plt.ylabel('F1-Score', fontsize=12)
    plt.legend(title='Task', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.ylim(0, 1.05)
    
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', rotation=90, fontsize=9)
    
    plt.tight_layout()
    save_plot('f1_score_comparison_all_tasks')
    plt.show()

In [51]:
# 3. Radar chart for best performing models
def create_radar_chart(data_dict: dict, title: str):
    """Create radar chart for model comparison."""
    from math import pi
    
    # Metrics to include
    metrics = ['Accuracy', 'F1_Score', 'Precision', 'Recall']
    
    # Number of metrics
    N = len(metrics)
    
    # Compute angle for each metric
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]  # Complete the circle
    
    # Initialize plot
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    # Plot data for each model
    colors = ['b', 'r', 'g', 'orange', 'purple', 'brown', 'pink']
    
    for i, (model, values) in enumerate(data_dict.items()):
        # Get values for this model
        model_values = [values.get(metric, 0) for metric in metrics]
        model_values += model_values[:1]  # Complete the circle
        
        # Plot
        ax.plot(angles, model_values, 'o-', linewidth=2, 
                label=model, color=colors[i % len(colors)])
        ax.fill(angles, model_values, alpha=0.25, color=colors[i % len(colors)])
    
    # Add metric labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metrics)
    
    # Set y-axis limits
    ax.set_ylim(0, 1)
    
    # Add title and legend
    plt.title(title, size=16, fontweight='bold', y=1.08)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    
    return fig, ax

# Create radar charts for each task
if not comparison_df.empty:
    for task in comparison_df['Task'].unique():
        task_data = comparison_df[comparison_df['Task'] == task]
        
        # Prepare data for radar chart
        radar_data = {}
        for _, row in task_data.iterrows():
            radar_data[row['Model']] = {
                'Accuracy': row['Accuracy'],
                'F1_Score': row['F1_Score'],
                'Precision': row['Precision'],
                'Recall': row['Recall']
            }
        
        if radar_data:
            fig, ax = create_radar_chart(radar_data, f'{task} - Model Performance')
            safe_task_name = task.lower().replace(' ', '_').replace('-', '_')
            save_plot(f'radar_chart_{safe_task_name}')
            plt.show()

## Best Model Analysis

In [52]:
# Identify best performing models for each task
if not comparison_df.empty:
    best_models_summary = []
    
    for task in comparison_df['Task'].unique():
        task_data = comparison_df[comparison_df['Task'] == task]
        
        # Find best model by accuracy
        best_by_accuracy = task_data.loc[task_data['Accuracy'].idxmax()]
        # Find best model by F1-score
        best_by_f1 = task_data.loc[task_data['F1_Score'].idxmax()]
        
        best_models_summary.append({
            'Task': task,
            'Best_by_Accuracy': best_by_accuracy['Model'],
            'Best_Accuracy': f"{best_by_accuracy['Accuracy']:.3f}",
            'Best_by_F1': best_by_f1['Model'],
            'Best_F1': f"{best_by_f1['F1_Score']:.3f}",
            'Avg_Accuracy': f"{task_data['Accuracy'].mean():.3f}",
            'Avg_F1': f"{task_data['F1_Score'].mean():.3f}"
        })
    
    best_models_df = pd.DataFrame(best_models_summary)
    save_dataframe(best_models_df, "best_models_summary", "comparison")
    
    print("\n=== Best Performing Models by Task ===")
    print(tabulate(best_models_df, headers="keys", tablefmt="grid"))

## Model Consistency Analysis

In [53]:
# Analyze which models perform consistently well across tasks
if not comparison_df.empty:
    # Calculate average performance across all tasks for each model
    model_consistency = comparison_df.groupby('Model').agg({
        'Accuracy': ['mean', 'std', 'count'],
        'F1_Score': ['mean', 'std'],
        'Precision': ['mean', 'std'],
        'Recall': ['mean', 'std']
    }).round(3)
    
    # Flatten column names
    model_consistency.columns = ['_'.join(col).strip() for col in model_consistency.columns]
    model_consistency = model_consistency.reset_index()
    
    # Sort by average accuracy
    model_consistency = model_consistency.sort_values('Accuracy_mean', ascending=False)
    
    save_dataframe(model_consistency, "model_consistency_analysis", "comparison")
    
    print("\n=== Model Consistency Analysis (Average Performance Across Tasks) ===")
    print(tabulate(model_consistency, headers="keys", tablefmt="grid", floatfmt=".3f"))
    
    # Plot consistency
    plt.figure(figsize=(12, 8))
    
    # Create bar plot with error bars
    x_pos = range(len(model_consistency))
    plt.bar(x_pos, model_consistency['Accuracy_mean'], 
            yerr=model_consistency['Accuracy_std'], 
            capsize=5, alpha=0.7)
    
    plt.xlabel('Model')
    plt.ylabel('Average Accuracy')
    plt.title('Model Consistency Across All Tasks\n(Error bars show standard deviation)', 
              fontsize=14, fontweight='bold')
    plt.xticks(x_pos, model_consistency['Model'], rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for i, (mean_val, std_val) in enumerate(zip(model_consistency['Accuracy_mean'], 
                                                model_consistency['Accuracy_std'])):
        plt.text(i, mean_val + std_val + 0.01, f'{mean_val:.3f}±{std_val:.3f}', 
                ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    save_plot('model_consistency_analysis')
    plt.show()

## Task Difficulty Analysis

In [54]:
# Analyze task difficulty based on average model performance
if not comparison_df.empty:
    task_difficulty = comparison_df.groupby('Task').agg({
        'Accuracy': ['mean', 'std', 'min', 'max'],
        'F1_Score': ['mean', 'std', 'min', 'max']
    }).round(3)
    
    task_difficulty.columns = ['_'.join(col).strip() for col in task_difficulty.columns]
    task_difficulty = task_difficulty.reset_index()
    
    # Sort by average accuracy (ascending = more difficult tasks first)
    task_difficulty = task_difficulty.sort_values('Accuracy_mean')
    
    save_dataframe(task_difficulty, "task_difficulty_analysis", "comparison")
    
    print("\n=== Task Difficulty Analysis (Based on Average Model Performance) ===")
    print(tabulate(task_difficulty, headers="keys", tablefmt="grid", floatfmt=".3f"))
    
    # Plot task difficulty
    plt.figure(figsize=(12, 6))
    
    x_pos = range(len(task_difficulty))
    plt.bar(x_pos, task_difficulty['Accuracy_mean'], 
            yerr=task_difficulty['Accuracy_std'], 
            capsize=5, alpha=0.7, color=['red', 'orange', 'green'])
    
    plt.xlabel('Task')
    plt.ylabel('Average Accuracy Across All Models')
    plt.title('Task Difficulty Analysis\n(Lower average accuracy indicates higher difficulty)', 
              fontsize=14, fontweight='bold')
    plt.xticks(x_pos, task_difficulty['Task'], rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for i, (mean_val, std_val) in enumerate(zip(task_difficulty['Accuracy_mean'], 
                                                task_difficulty['Accuracy_std'])):
        plt.text(i, mean_val + std_val + 0.01, f'{mean_val:.3f}±{std_val:.3f}', 
                ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    save_plot('task_difficulty_analysis')
    plt.show()

## Comprehensive Summary Report

In [55]:
# Create comprehensive summary report
summary_report = {
    'project_info': {
        'name': 'Tennessee Eastman Process Analysis',
        'version': VERSION,
        'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'total_notebooks': 6,
        'completed_notebooks': 5  # Data prep, EDA, Classification, Anomaly, Merging
    },
    'data_summary': {
        'dataset': 'Tennessee Eastman Process',
        'features': 52,  # Assuming from typical TE dataset
        'analysis_types': ['Multi-class Classification', 'Binary Anomaly Detection', 'Merged Fault Classification']
    },
    'model_summary': {
        'total_models_trained': len(comparison_df['Model'].unique()) if not comparison_df.empty else 0,
        'model_types': list(comparison_df['Model'].unique()) if not comparison_df.empty else [],
        'best_overall_model': model_consistency.iloc[0]['Model'] if not comparison_df.empty else 'N/A',
        'most_consistent_model': model_consistency.iloc[0]['Model'] if not comparison_df.empty else 'N/A'
    },
    'performance_summary': {
        'highest_accuracy': comparison_df['Accuracy'].max() if not comparison_df.empty else 0,
        'highest_f1_score': comparison_df['F1_Score'].max() if not comparison_df.empty else 0,
        'average_accuracy_across_all': comparison_df['Accuracy'].mean() if not comparison_df.empty else 0,
        'average_f1_across_all': comparison_df['F1_Score'].mean() if not comparison_df.empty else 0
    },
    'task_specific_summaries': {
        'classification': classification_summary,
        'anomaly_detection': anomaly_summary,
        'merged_faults': merged_summary
    }
}

# Save comprehensive summary
summary_path = os.path.join(OUTPUT_PATH, VERSION, f"comprehensive_analysis_summary_v{VERSION}_.json")
with open(summary_path, 'w') as f:
    json.dump(summary_report, f, indent=2, default=str)

print("\n" + "="*80)
print("\t\tTENNESSEE EASTMAN PROCESS - COMPREHENSIVE ANALYSIS SUMMARY")
print("="*80)

print(f"\n📊 PROJECT INFORMATION:")
print(f"   • Analysis Version: {summary_report['project_info']['version']}")
print(f"   • Analysis Date: {summary_report['project_info']['analysis_date']}")
print(f"   • Total Notebooks: {summary_report['project_info']['total_notebooks']}")
print(f"   • Completed Analyses: {summary_report['project_info']['completed_notebooks']}")

print(f"\n🔬 ANALYSIS SCOPE:")
print(f"   • Dataset: {summary_report['data_summary']['dataset']}")
print(f"   • Features: {summary_report['data_summary']['features']}")
print(f"   • Analysis Types: {', '.join(summary_report['data_summary']['analysis_types'])}")

print(f"\n🤖 MODEL SUMMARY:")
print(f"   • Total Models Trained: {summary_report['model_summary']['total_models_trained']}")
print(f"   • Model Types: {', '.join(summary_report['model_summary']['model_types'])}")
print(f"   • Best Overall Model: {summary_report['model_summary']['best_overall_model']}")

print(f"\n📈 PERFORMANCE HIGHLIGHTS:")
print(f"   • Highest Accuracy: {summary_report['performance_summary']['highest_accuracy']:.3f}")
print(f"   • Highest F1-Score: {summary_report['performance_summary']['highest_f1_score']:.3f}")
print(f"   • Average Accuracy: {summary_report['performance_summary']['average_accuracy_across_all']:.3f}")
print(f"   • Average F1-Score: {summary_report['performance_summary']['average_f1_across_all']:.3f}")

print(f"\n📁 OUTPUT FILES GENERATED:")
print(f"   • Location: {OUTPUT_PATH}/{VERSION}/")
print(f"   • Comprehensive Summary: comprehensive_analysis_summary_v{VERSION}_.json")
print(f"   • Cross-Task Comparison: cross_task_comparison_comparison_v{VERSION}_.csv")
print(f"   • Best Models Summary: best_models_summary_comparison_v{VERSION}_.csv")
print(f"   • Model Consistency Analysis: model_consistency_analysis_comparison_v{VERSION}_.csv")
print(f"   • Task Difficulty Analysis: task_difficulty_analysis_comparison_v{VERSION}_.csv")
print(f"   • Visualization Plots: comparison/ subfolder")

print("\n" + "="*80)
print("\t\t\t\tANALYSIS COMPLETE")
print("="*80)


		TENNESSEE EASTMAN PROCESS - COMPREHENSIVE ANALYSIS SUMMARY

📊 PROJECT INFORMATION:
   • Analysis Version: 1.00
   • Analysis Date: 2025-08-21 22:24:56
   • Total Notebooks: 6
   • Completed Analyses: 5

🔬 ANALYSIS SCOPE:
   • Dataset: Tennessee Eastman Process
   • Features: 52
   • Analysis Types: Multi-class Classification, Binary Anomaly Detection, Merged Fault Classification

🤖 MODEL SUMMARY:
   • Total Models Trained: 0
   • Model Types: 
   • Best Overall Model: N/A

📈 PERFORMANCE HIGHLIGHTS:
   • Highest Accuracy: 0.000
   • Highest F1-Score: 0.000
   • Average Accuracy: 0.000
   • Average F1-Score: 0.000

📁 OUTPUT FILES GENERATED:
   • Location: output/1.00/
   • Comprehensive Summary: comprehensive_analysis_summary_v1.00_.json
   • Cross-Task Comparison: cross_task_comparison_comparison_v1.00_.csv
   • Best Models Summary: best_models_summary_comparison_v1.00_.csv
   • Model Consistency Analysis: model_consistency_analysis_comparison_v1.00_.csv
   • Task Difficulty Analysis

## Key Insights and Recommendations

In [56]:
print("\n🔍 KEY INSIGHTS:")
print("="*50)

if not comparison_df.empty:
    # Task-specific insights
    tasks = comparison_df['Task'].unique()
    
    for task in tasks:
        task_data = comparison_df[comparison_df['Task'] == task]
        best_model = task_data.loc[task_data['Accuracy'].idxmax(), 'Model']
        best_accuracy = task_data['Accuracy'].max()
        worst_accuracy = task_data['Accuracy'].min()
        
        print(f"\n• {task}:")
        print(f"  - Best Model: {best_model} ({best_accuracy:.3f} accuracy)")
        print(f"  - Performance Range: {worst_accuracy:.3f} - {best_accuracy:.3f}")
        print(f"  - Performance Spread: {best_accuracy - worst_accuracy:.3f}")
    
    # Overall insights
    most_consistent = model_consistency.iloc[0]['Model']
    print(f"\n• Overall Best Performing Model: {most_consistent}")
    print(f"• Most Challenging Task: {task_difficulty.iloc[0]['Task']}")
    print(f"• Easiest Task: {task_difficulty.iloc[-1]['Task']}")

print("\n💡 RECOMMENDATIONS:")
print("="*50)
print("• For Production Deployment: Use the most consistent model across tasks")
print("• For Specific Tasks: Use task-specific best performing models")
print("• For Further Research: Focus on improving performance for the most challenging task")
print("• Model Selection: Consider ensemble methods combining top performers")
print("• Data Quality: Investigate why certain tasks are more challenging")

print("\n✅ NEXT STEPS:")
print("="*50)
print("1. Review individual notebook results for detailed analysis")
print("2. Consider hyperparameter tuning for best performing models")
print("3. Implement ensemble methods for improved performance")
print("4. Analyze feature importance across different tasks")
print("5. Consider domain-specific feature engineering")
print("6. Validate results with additional cross-validation")
print("7. Deploy selected models for real-time monitoring")


🔍 KEY INSIGHTS:

💡 RECOMMENDATIONS:
• For Production Deployment: Use the most consistent model across tasks
• For Specific Tasks: Use task-specific best performing models
• For Further Research: Focus on improving performance for the most challenging task
• Model Selection: Consider ensemble methods combining top performers
• Data Quality: Investigate why certain tasks are more challenging

✅ NEXT STEPS:
1. Review individual notebook results for detailed analysis
2. Consider hyperparameter tuning for best performing models
3. Implement ensemble methods for improved performance
4. Analyze feature importance across different tasks
5. Consider domain-specific feature engineering
6. Validate results with additional cross-validation
7. Deploy selected models for real-time monitoring
